# import Library

In [1]:
import pandas as pd
from pandas import DataFrame as df

import numpy as np
import re
from datetime import datetime
import dbfread


import descartes
#!pip install geopandas
import geopandas as gpd
#!pip install shapely
from shapely.geometry import Point, Polygon
from math import sin,cos,sqrt,atan2,radians 

import requests
import logging
import time
#!pip install googlemaps
import googlemaps
#!pip install reverse_geocoder
import reverse_geocoder
#!pip install pprint
import pprint
#!pip install folium
#!pip install pygeocoder
#import pygeocoder
import folium
import zipfile
import json
#%matplotlib.inline


import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline

matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호도 표시

# 한글 깨짐 방지 목적 #
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family = font_name)
rc('font', family='NanumGothic')

# !pip install fbprophet
# from fbprophet.plot import plot_plotly
# import plotly.offline as py

import pandas_profiling

import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 숨길 때

# Import data

## Merging을 위해 유통 관련 변수들은 제거한다.

In [2]:
df = pd.read_csv('가공데이터/merge_유통_유동_환경_소상공인_edit1.csv')
df = df.iloc[:,1:]
df['date'] = pd.to_datetime(df['date'],format = '%Y-%m-%d')
# 유통관련 변수들 제거
df = df[df.columns[:-26]]

card = pd.read_csv('Rawdata/카드매출데이터/CARD_SPENDING_190809.txt', sep = "\t",engine='python',encoding='euc-kr')
card.GU_CD = card.GU_CD.astype('str')
card.DONG_CD = card.DONG_CD.astype('str')

# 카드매출데이터 전처리

In [3]:
GU = '''
110	515	종로구	청운효자동
110	530	종로구	사직동
110	540	종로구	삼청동
110	550	종로구	부암동
110	560	종로구	평창동
110	570	종로구	무악동
110	580	종로구	교남동
110	600	종로구	가회동
110	615	종로구	종로1.2.3.4가동
110	630	종로구	종로5.6가동
110	640	종로구	이화동
110	650	종로구	혜화동
110	670	종로구	창신1동
110	680	종로구	창신2동
110	690	종로구	창신3동
110	700	종로구	숭인1동
110	710	종로구	숭인2동
350	560	노원구	월계1동
350	570	노원구	월계2동
350	580	노원구	월계3동
350	595	노원구	공릉1동
350	600	노원구	공릉2동
350	611	노원구	하계1동
350	612	노원구	하계2동
350	619	노원구	중계본동
350	621	노원구	중계1동
350	624	노원구	중계4동
350	625	노원구	중계2.3동
350	630	노원구	상계1동
350	640	노원구	상계2동
350	665	노원구	상계3.4동
350	670	노원구	상계5동
350	695	노원구	상계6.7동
350	700	노원구	상계8동
350	710	노원구	상계9동
350	720	노원구	상계10동
'''

In [4]:
df_GU=pd.DataFrame(re.findall(r"[\S]+",GU))

In [5]:
df_GU=pd.DataFrame(df_GU.values.reshape(36,4))

In [6]:
df_GU.rename(columns={0:"GU_CD",1:"DONG_CD",2:"GU",3:"DONG"}, inplace=True)

In [7]:
df_GU['GU_CD']=df_GU['GU_CD'].astype('category')
df_GU['DONG_CD']=df_GU['DONG_CD'].astype('category')
df_GU['GU']=df_GU['GU'].astype('category')
df_GU['DONG']=df_GU['DONG'].astype('category')

In [8]:
list = pd.DataFrame(['숙박(10)', '레저용품(20)', '레저업소(21)', '문화취미(22)', '가구(30)', '전기(31)', '주방용구(32)', '연료판매(33)', '광학제품(34)', '가전(35)', '유통업(40)', '의복(42)', '직물(43)', '신변잡화(44)', '서적문구(50)', '사무통신(52)', '자동차판매(60)', '자동차정비(62)', '의료기관(70)', '보건위생(71)', '요식업소(80)', '음료식품(81)', '수리서비스(92)'], columns=['MCT_CAT_CD'])

In [9]:
cate=list['MCT_CAT_CD'].str.split("(", n=1, expand=True)

In [10]:
cate.rename({0:'NAME',1:'MCT_CAT_CD'},axis=1,inplace=True)
cate['MCT_CAT_CD']=cate['MCT_CAT_CD'].str.replace(")","")

In [11]:
cate['NAME']=cate['NAME'].astype('category')
cate['MCT_CAT_CD']=cate['MCT_CAT_CD'].astype('category')

In [12]:
df_GU.GU_CD = df_GU.GU_CD.astype('str')
df_GU.DONG_CD = df_GU.DONG_CD.astype('str')

In [13]:
df_GU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
GU_CD      36 non-null object
DONG_CD    36 non-null object
GU         36 non-null category
DONG       36 non-null category
dtypes: category(2), object(2)
memory usage: 2.3+ KB


In [14]:
card_gd = card.merge(df_GU,how='left', on = ['GU_CD','DONG_CD'])

In [15]:
card_gd.isna().sum()

STD_DD        0
GU_CD         0
DONG_CD       0
MCT_CAT_CD    0
SEX_CD        0
AGE_CD        0
USE_CNT       0
USE_AMT       0
GU            0
DONG          0
dtype: int64

In [16]:
card_gd.columns = ['STD_YMD', 'GU_CD', 'DONG_CD', 'MCT_CAT_CD', 'SEX_CD', 'AGE_CD',
       'USE_CNT', 'USE_AMT', 'GU', 'ADMD_NM']

In [17]:
card_gd.GU = card_gd.GU.astype('str')
card_gd.ADMD_NM = card_gd.ADMD_NM.astype('str')

In [18]:
card_gd = card_gd[['STD_YMD', 'MCT_CAT_CD', 'SEX_CD', 'AGE_CD','USE_CNT', 'USE_AMT', 'ADMD_NM']]

In [19]:
card_wide = card_gd.pivot_table(index=['STD_YMD','ADMD_NM'], columns=['SEX_CD','AGE_CD','MCT_CAT_CD' ], aggfunc= lambda x: x)

In [20]:
card_wide = card_wide.fillna(0)

In [21]:
card_wide.head()

USE_AMT                                                 ...  \
SEX_CD                 F                                                 ...   
AGE_CD                20                                                 ...   
MCT_CAT_CD            10   20     21   22   30   31   32   33   34   35  ...   
STD_YMD  ADMD_NM                                                         ...   
20180401 가회동       173.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
         공릉1동      136.0  0.0  374.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
         공릉2동        0.0  0.0  305.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
         교남동         0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
         무악동         0.0  0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

                 USE_CNT                                                     
SEX_CD                 M                                                     
AGE_CD                65                                                     
MCT_CAT_CD            44   50   52   60    62    70    71     80    81   92  
STD_YMD  ADMD_NM                                                             
20180401 가회동         0.0  0.0  0.0  0.0   0.0   0.0   0.0   26.0   0.0  0.0  
         공릉1동        0.0  0.0  0.0  0.0   0.0   4.0  13.0  175.0  31.0  0.0  
         공릉2동        0.0  0.0  0.0  0.0   4.0   4.0  31.0   39.0  18.0  0.0  
         교남동         0.0  0.0  0.0  0.0  13.0  18.0   0.0   57.0  26.0  0.0  
         무악동         0.0  0.0  0.0  0.0   0.0   4.0   0.0   13.0   9.0  0.0  

[5 rows x 920 columns]

In [22]:
card_wide.columns.ravel()

array([('USE_AMT', 'F', 20, 10), ('USE_AMT', 'F', 20, 20),
       ('USE_AMT', 'F', 20, 21), ('USE_AMT', 'F', 20, 22),
       ('USE_AMT', 'F', 20, 30), ('USE_AMT', 'F', 20, 31),
       ('USE_AMT', 'F', 20, 32), ('USE_AMT', 'F', 20, 33),
       ('USE_AMT', 'F', 20, 34), ('USE_AMT', 'F', 20, 35),
       ('USE_AMT', 'F', 20, 40), ('USE_AMT', 'F', 20, 42),
       ('USE_AMT', 'F', 20, 43), ('USE_AMT', 'F', 20, 44),
       ('USE_AMT', 'F', 20, 50), ('USE_AMT', 'F', 20, 52),
       ('USE_AMT', 'F', 20, 60), ('USE_AMT', 'F', 20, 62),
       ('USE_AMT', 'F', 20, 70), ('USE_AMT', 'F', 20, 71),
       ('USE_AMT', 'F', 20, 80), ('USE_AMT', 'F', 20, 81),
       ('USE_AMT', 'F', 20, 92), ('USE_AMT', 'F', 25, 10),
       ('USE_AMT', 'F', 25, 20), ('USE_AMT', 'F', 25, 21),
       ('USE_AMT', 'F', 25, 22), ('USE_AMT', 'F', 25, 30),
       ('USE_AMT', 'F', 25, 31), ('USE_AMT', 'F', 25, 32),
       ('USE_AMT', 'F', 25, 33), ('USE_AMT', 'F', 25, 34),
       ('USE_AMT', 'F', 25, 35), ('USE_AMT', 'F', 25, 40

In [23]:
card_wide.columns = ['_'.join((str(col[3])+'M', str(col[2])+'A', str(col[1]), str(col[0]))) for col in card_wide.columns.ravel()]

In [24]:
card_wide.reset_index(inplace=True)

In [25]:
card_wide.STD_YMD = card_wide.STD_YMD.astype('str')
card_wide['STD_YMD'] = pd.to_datetime(card_wide['STD_YMD'],format = '%Y%m%d')
card_wide = card_wide.rename(columns = {'STD_YMD':'date',
                                        'ADMD_NM':'district'})

In [26]:
card_wide

,date,district,10M_20A_F_USE_AMT,20M_20A_F_USE_AMT,21M_20A_F_USE_AMT,22M_20A_F_USE_AMT,30M_20A_F_USE_AMT,31M_20A_F_USE_AMT,32M_20A_F_USE_AMT,33M_20A_F_USE_AMT,...,44M_65A_M_USE_CNT,50M_65A_M_USE_CNT,52M_65A_M_USE_CNT,60M_65A_M_USE_CNT,62M_65A_M_USE_CNT,70M_65A_M_USE_CNT,71M_65A_M_USE_CNT,80M_65A_M_USE_CNT,81M_65A_M_USE_CNT,92M_65A_M_USE_CNT
0,2018-04-01,가회동,173.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0
1,2018-04-01,공릉1동,136.0,0.0,374.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,13.0,175.0,31.0,0.0
2,2018-04-01,공릉2동,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,4.0,31.0,39.0,18.0,0.0
3,2018-04-01,교남동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,18.0,0.0,57.0,26.0,0.0
4,2018-04-01,무악동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,13.0,9.0,0.0
5,2018-04-01,부암동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,22.0,0.0,0.0
6,2018-04-01,사직동,0.0,0.0,148.0,958.0,0.0,0.0,0.0,18.0,...,0.0,4.0,0.0,0.0,61.0,0.0,0.0,298.0,22.0,0.0
7,2018-04-01,삼청동,0.0,0.0,0.0,947.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,4.0,0.0
8,2018-04-01,상계10동,0.0,0.0,664.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2018-04-01,상계1동,0.0,131.0,193.0,210.0,0.0,0.0,0.0,131.0,...,0.0,13.0,0.0,0.0,0.0,4.0,35.0,302.0,26.0,0.0


In [27]:
merging = pd.merge(df,card_wide,on = ['date','district'])

In [28]:
merging.to_csv('가공데이터/merge_카드_유동_환경_edit2.csv')